# Comparación Multipunto de la Frontera de Diversificación

Este notebook evalúa múltiples puntos relevantes de la frontera de diversificación y optimiza carteras para cada uno, usando el mismo pipeline del módulo de selección de activos.

In [ ]:
import sys
from pathlib import Path
import importlib
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# Añadir src/ al path si es necesario
src_path = Path('..') / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Imports de módulos necesarios
datos = importlib.import_module('1datos')
diversificacion = importlib.import_module('2equiponderada_diversificacion')
multipunto = importlib.import_module('5analisis_multipunto')
seleccion = importlib.import_module('4seleccion_activos')

print('Librerías y módulos importados correctamente')

## Cargar datos

Se cargan los retornos diarios desde el archivo de datos y se valida la dimensión del DataFrame.

In [ ]:
ruta_csv = Path('..') / 'data' / 'retornos_diarios.csv'
retornos = datos.cargar_retornos(str(ruta_csv))

print(f"Dimensiones del DataFrame: {retornos.shape[0]} días, {retornos.shape[1]} activos")
print(retornos.head())

## Simular frontera de diversificación

Se simula la frontera equiponderada y se visualizan los resultados de riesgo total y descomposición.

In [ ]:
df_frontera = diversificacion.simular_frontera_diversificacion(
    retornos,
    n_valores=None,
    n_simulaciones=100
)

print(df_frontera.head())

ruta_frontera = Path('..') / 'outputs' / 'frontera_diversificacion_multipunto.png'
fig = diversificacion.visualizar_frontera_diversificacion(df_frontera, ruta_guardado=str(ruta_frontera))
plt.show()

## Detectar puntos de interés

Se detectan mínimos locales, puntos con reducción marginal baja y cambios de pendiente relevantes.

In [ ]:
criterios = {
    'umbral_reduccion': 2.0,
    'umbral_cambio_pendiente': None
}

lista_n = multipunto.detectar_puntos_interes_frontera(df_frontera, criterios)
print(f"Puntos de interés detectados: {lista_n}")

# Explicación breve de relevancia para cada N
vol = df_frontera['volatilidad_media'].values
n_vals = df_frontera['n_activos'].astype(int).values
reduccion = df_frontera['reduccion_pct'].values
pendientes = np.diff(vol) / np.diff(n_vals)
cambios_pendiente = np.diff(pendientes) if len(pendientes) > 1 else np.array([])

umbral_reduccion = criterios['umbral_reduccion']
umbral_cambio = criterios['umbral_cambio_pendiente']
if umbral_cambio is None:
    umbral_cambio = np.std(cambios_pendiente) if len(cambios_pendiente) > 0 else 0.0

for n in lista_n:
    idx = np.where(n_vals == n)[0][0]
    razones = []
    if 0 < idx < len(vol) - 1 and vol[idx] < vol[idx - 1] and vol[idx] < vol[idx + 1]:
        razones.append('mínimo local de volatilidad')
    if not np.isnan(reduccion[idx]) and reduccion[idx] < umbral_reduccion:
        razones.append(f"reducción marginal < {umbral_reduccion}%")
    if len(cambios_pendiente) > 0 and 1 <= idx < len(n_vals) - 1:
        cambio_idx = idx - 1
        if 0 <= cambio_idx < len(cambios_pendiente) and abs(cambios_pendiente[cambio_idx]) > umbral_cambio:
            razones.append('cambio de pendiente significativo')
    if not razones:
        razones.append('punto de control')
    print(f"N={n}: {', '.join(razones)}")

## Optimizar carteras para todos los puntos

Se ejecuta la optimización completa para cada N detectado.

In [ ]:
resultados = multipunto.optimizar_multiples_n(
    retornos,
    lista_n=lista_n,
    rf_anual=0.02,
    peso_sharpe=0.7
)

## Consolidar y mostrar resultados

Se consolida la información de todas las carteras para comparar Sharpe, rentabilidad y volatilidad.

In [ ]:
df_consolidado = multipunto.consolidar_resultados_multipunto(resultados)
print(df_consolidado)

mejor_n = int(df_consolidado.loc[0, 'n_activos'])
mejor_sharpe = df_consolidado.loc[0, 'sharpe_cartera']
print(f"Mejor Sharpe encontrado en N={mejor_n} con Sharpe={mejor_sharpe:.4f}")

## Visualización comparativa

Se comparan las carteras en un panel 2x2 con Sharpe, riesgo-retorno, rentabilidad vs volatilidad y peso en RF.

In [ ]:
ruta_comparacion = Path('..') / 'outputs' / 'comparacion_multipunto'
fig = multipunto.visualizar_comparacion_multipunto(df_consolidado, resultados, ruta_guardado=str(ruta_comparacion))
plt.show()

## Heatmap de composición

Se muestra un heatmap con los pesos de los activos más utilizados en las carteras.

In [ ]:
ruta_heatmap = Path('..') / 'outputs' / 'heatmap_pesos_multipunto'
fig = multipunto.generar_heatmap_pesos(resultados, top_activos=15, ruta_guardado=str(ruta_heatmap))
plt.show()

## Análisis detallado de la mejor cartera

Se revisa la composición y las métricas de la cartera con mejor Sharpe, junto con una comparación frente al baseline equiponderado.

In [ ]:
resultado_mejor = resultados[mejor_n]

pesos_completos = resultado_mejor['pesos_completos']
activos_labels = [f'asset{i+1}' for i in range(len(pesos_completos))]

df_pesos_mejor = pd.DataFrame({
    'activo': activos_labels,
    'peso': pesos_completos
}).sort_values('peso', ascending=False).reset_index(drop=True)

print("Activos seleccionados (peso > 0):")
print(df_pesos_mejor[df_pesos_mejor['peso'] > 0].head(20))

print("\nMétricas completas:")
print(resultado_mejor['metricas_cartera'])

print("\nComparación con baseline equiponderado:")
print(resultado_mejor['comparacion_baseline'])

# Gráfico de pesos de la mejor cartera
plt.figure(figsize=(12, 4))
plt.bar(df_pesos_mejor['activo'], df_pesos_mejor['peso'] * 100, color='steelblue', edgecolor='black')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Peso (%)')
plt.title(f'Pesos de la Cartera con Mejor Sharpe (N={mejor_n})')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Análisis detallado de otras carteras destacadas

Se comparan las 2-3 carteras con mejor Sharpe después de la mejor.8

In [ ]:
top_k = 4
mejores = df_consolidado.head(top_k)['n_activos'].astype(int).tolist()
mejores = [n for n in mejores if n != mejor_n][:3]

for n in mejores:
    resultado = resultados[n]
    metricas = resultado['metricas_cartera']
    activos_labels = [f'asset{i+1}' for i in range(len(resultado['pesos_completos']))]
    df_pesos = pd.DataFrame({
        'activo': activos_labels,
        'peso': resultado['pesos_completos']
    }).sort_values('peso', ascending=False)

    print(f"\nCartera N={n}")
    print(f"Sharpe: {metricas['sharpe']:.4f} | Rentabilidad: {metricas['rentabilidad']*100:.2f}% | Volatilidad: {metricas['volatilidad']*100:.2f}%")
    print("Top activos por peso:")
    print(df_pesos.head(10))

## Conclusiones

- El N óptimo según estos resultados es el que maximiza el Sharpe en la tabla consolidada.
- Se observa el trade-off entre diversificación y rendimiento al comparar rentabilidad, volatilidad y Sharpe.
- El peso en renta fija permite verificar si hay una tendencia sistemática entre N y asignación a RF.
- Recomendación: usar la cartera con mejor Sharpe si la reducción de volatilidad no sacrifica demasiado rendimiento.